In [1]:
from sudoku import Sudoku
import pandas as pd
import subprocess

In [2]:
#get main cnf file
path = 'sudoku9.cnf'
cnf =""
f = open(path, 'r')
for line in f:
    cnf+= line
f.close()

In [3]:
# Global Constants
MIN_VARS = 1
MAX_VARS = 25
INSTANCES_PER_VAR = 1
SUDOKU_DEGREE = 9
VAR_COUNT = int(str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE) + str(SUDOKU_DEGREE))
SOLVER_NAME = "minisat"

In [4]:
frames = [] # will store dataframes for each variable

In [5]:
frames = []
for var in range(MIN_VARS, MAX_VARS):
    boards = []
    frame = pd.DataFrame(columns=['board', 'solution'])
    for y in range(INSTANCES_PER_VAR):
        board = Sudoku(SUDOKU_DEGREE)
        board.randomize_board(var)
        boards.append(board)
    cnfs = [board.cnf_output() for board in boards]
    frame['board'] = boards
    frame['cnf'] = cnfs
    frame['num_vars'] = var
    frames.append(frame)
frames = pd.DataFrame(pd.concat(frames))
frames.head(5)

,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7ff7cf1b44a8>,NaN,259 0\n,1
0,<sudoku.Sudoku object at 0x7ff7aad2df60>,NaN,033 0\n163 0\n,2
0,<sudoku.Sudoku object at 0x7ff7aad35160>,NaN,402 0\n516 0\n762 0\n,3
0,<sudoku.Sudoku object at 0x7ff7aad35400>,NaN,173 0\n466 0\n537 0\n613 0\n,4
0,<sudoku.Sudoku object at 0x7ff7aad356d8>,NaN,319 0\n346 0\n586 0\n687 0\n778 0\n,5


In [6]:
# This function writes a DIMACS suited file using a string in cnf
def write_cnf_file(fname, string):
    
    # count the number of lines in the string
    clause_count = len(string.splitlines())
    
    # writes the file back with the suitable header
    with open(fname, "w") as f:
        f.write("p cnf " + str(VAR_COUNT) + " " + str(clause_count) + "\n" + string)
        
    return

In [7]:
# This function solves a DIMACS file using a SAT solver and bash command
def solve_using_bash(input_f, output_f):
    
    # the bash command we feed to the terminal
    bashCommand = SOLVER_NAME + " " + input_f + " " + output_f
    
    # where the magic happens
    process = subprocess.Popen(bashCommand.split(), stdout = subprocess.PIPE)
    
    # print any errors to stdout
    output, error = process.communicate()
    
    return

In [8]:
def get_solution_from_cnf(cnf_string):
    
    cnf_string = cnf + cnf_string #append the whole cnf
    
     # Our temporary input/output files
    input_file = "inputtemp.cnf"
    output_file = "outputtemp.txt"
    
    write_cnf_file(input_file, cnf_string)
    
    solve_using_bash(input_file, output_file)
    
    # open the output and send the contents 
    with open (output_file) as f:
        return f.read()

In [9]:
%%time
frames['solution'] = frames['cnf'].apply(get_solution_from_cnf)

CPU times: user 36 ms, sys: 80 ms, total: 116 ms
Wall time: 4.34 s


In [10]:
frames.head(5)



,board,solution,cnf,num_vars
0,<sudoku.Sudoku object at 0x7ff7cf1b44a8>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,259 0\n,1
0,<sudoku.Sudoku object at 0x7ff7aad2df60>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,033 0\n163 0\n,2
0,<sudoku.Sudoku object at 0x7ff7aad35160>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,402 0\n516 0\n762 0\n,3
0,<sudoku.Sudoku object at 0x7ff7aad35400>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,173 0\n466 0\n537 0\n613 0\n,4
0,<sudoku.Sudoku object at 0x7ff7aad356d8>,SAT\n-1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -1...,319 0\n346 0\n586 0\n687 0\n778 0\n,5
